In [1]:
# 필수(?) 라이브러리 호출
import pandas as pd
import numpy as np
import math

In [2]:
# 경고가 많아지면 곤란해서 추가
import warnings
warnings.filterwarnings('ignore')

In [3]:
# 일단 분석할 데이터를 가져오는데
df = pd.read_csv("../raw_data/all_data_test2.csv", encoding="CP949").reset_index(drop=True)
df

,회사명,거래소코드,회계년도,영업이익(손실)(천원),이자비용(천원),영업이익(손실)(연결)(천원),이자비용(연결)(천원),계속영업이익(손실)(IFRS)(천원),이자비용(IFRS)(천원),계속영업이익(손실)(IFRS연결)(천원),이자비용(IFRS연결)(천원),[제조]이자보상배율(이자비용),[제조]이자보상배율(이자비용)(IFRS연결),[제조]영업이익증가율,[제조]재고자산증가율,[제조]자기자본증가율,[제조]매출액증가율,[제조]매출액총이익률,[제조]부가가치율,[제조]노동소득분배율
0,(주)CMG제약,58820,2000/12,1534583.0,307112.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,(주)CMG제약,58820,2001/12,294030.0,284906.0,NaN,NaN,NaN,NaN,NaN,NaN,1.03,NaN,0.00,0.00,0.00,0.00,5.53,11.57,65.34
2,(주)CMG제약,58820,2002/12,-3817019.0,382103.0,NaN,NaN,NaN,NaN,NaN,NaN,-9.99,NaN,0.00,-39.91,-25.65,66.81,-10.12,0.46,2111.99
3,(주)CMG제약,58820,2003/12,-914055.0,271348.0,NaN,NaN,NaN,NaN,NaN,NaN,-3.37,NaN,0.00,-11.75,-1.55,-34.12,0.70,10.69,63.75
4,(주)CMG제약,58820,2004/12,-2838322.0,123816.0,NaN,NaN,NaN,NaN,NaN,NaN,-22.92,NaN,0.00,-53.72,-60.20,-24.76,-14.51,-15.02,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32966,흥아해운(주),3280,2015/12,16258441.0,20637952.0,21244278.0,20754125.0,8827312.0,20637952.0,10517216.0,20754125.0,0.79,1.02,2.53,-12.92,5.12,1.56,4.73,7.33,25.11
32967,흥아해운(주),3280,2016/12,1154122.0,27547679.0,5895873.0,27640109.0,-16149442.0,27547679.0,-17173253.0,27640109.0,0.04,0.21,-92.90,56.39,13.60,-2.69,2.95,6.07,31.80
32968,흥아해운(주),3280,2017/12,-18649057.0,30806949.0,-13098624.0,30888216.0,-67916740.0,30806949.0,-73500411.0,30888216.0,-0.61,-0.42,0.00,11.18,-41.46,0.11,0.36,0.43,444.12
32969,흥아해운(주),3280,2018/12,-40598236.0,32230077.0,-37595967.0,32310603.0,-96953501.0,32230077.0,-86540156.0,32310603.0,-1.26,-1.16,0.00,-5.92,-47.28,-9.71,-2.77,-5.34,0.00


In [4]:
df2 = df.drop(columns=['거래소코드', '계속영업이익(손실)(IFRS)(천원)', '이자비용(IFRS)(천원)', '계속영업이익(손실)(IFRS연결)(천원)', '이자비용(IFRS연결)(천원)',
'[제조]이자보상배율(이자비용)', '[제조]이자보상배율(이자비용)(IFRS연결)'])

In [5]:
df2

,회사명,회계년도,영업이익(손실)(천원),이자비용(천원),영업이익(손실)(연결)(천원),이자비용(연결)(천원),[제조]영업이익증가율,[제조]재고자산증가율,[제조]자기자본증가율,[제조]매출액증가율,[제조]매출액총이익률,[제조]부가가치율,[제조]노동소득분배율
0,(주)CMG제약,2000/12,1534583.0,307112.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,(주)CMG제약,2001/12,294030.0,284906.0,NaN,NaN,0.00,0.00,0.00,0.00,5.53,11.57,65.34
2,(주)CMG제약,2002/12,-3817019.0,382103.0,NaN,NaN,0.00,-39.91,-25.65,66.81,-10.12,0.46,2111.99
3,(주)CMG제약,2003/12,-914055.0,271348.0,NaN,NaN,0.00,-11.75,-1.55,-34.12,0.70,10.69,63.75
4,(주)CMG제약,2004/12,-2838322.0,123816.0,NaN,NaN,0.00,-53.72,-60.20,-24.76,-14.51,-15.02,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32966,흥아해운(주),2015/12,16258441.0,20637952.0,21244278.0,20754125.0,2.53,-12.92,5.12,1.56,4.73,7.33,25.11
32967,흥아해운(주),2016/12,1154122.0,27547679.0,5895873.0,27640109.0,-92.90,56.39,13.60,-2.69,2.95,6.07,31.80
32968,흥아해운(주),2017/12,-18649057.0,30806949.0,-13098624.0,30888216.0,0.00,11.18,-41.46,0.11,0.36,0.43,444.12
32969,흥아해운(주),2018/12,-40598236.0,32230077.0,-37595967.0,32310603.0,0.00,-5.92,-47.28,-9.71,-2.77,-5.34,0.00


In [6]:
# 이.보.배 계산

df2['이자보상배율(이자비용)'] = df2['영업이익(손실)(천원)']/df2['이자비용(천원)']
df2['이자보상배율(이자비용)(IFRS연결)'] = df2['영업이익(손실)(연결)(천원)']/df2['이자비용(연결)(천원)']

In [7]:
df2['이자보상배율(이자비용)(IFRS연결)'].value_counts()

 inf          456
-inf           88
 8.992248       1
 13.319852      1
 2.619002       1
             ... 
-25.966350      1
 1.401843       1
-0.029851       1
-0.811271       1
-0.549389       1
Name: 이자보상배율(이자비용)(IFRS연결), Length: 16608, dtype: int64

In [8]:
# 분석 과정에서 NaN이 NaN이 아닌 다른 어떤 값인 것으로 판명되었습니다.
# 어쩔 수 없이 분석의 용이함을 위해 임의의 값을 채웠습니다
df2['이자보상배율(이자비용)'] = df2['이자보상배율(이자비용)'].fillna(77777777.77)
df2['이자보상배율(이자비용)(IFRS연결)'] = df2['이자보상배율(이자비용)(IFRS연결)'].fillna(77777777.77) # 이거 사실 GAAT 기준이긴한데...

# 이건 우리의 기준에 맞는 이자보상 배율을 저장하기 위한 컬럼입니다.
df2['이자보상배율'] = 0.0

In [9]:
# csv 한바퀴 돌립니다
for i in range(0,len(df2)):
    #print(i)

    # 구 기준 회계값에 결측치가 없다면
    if df2.iloc[i]['이자보상배율(이자비용)'] != 77777777.77:
        # 신 기준 회계값에도 결측치가 없다면
        if df2.iloc[i]['이자보상배율(이자비용)(IFRS연결)'] != 77777777.77:
            # 신 회계값 삽입
            df2.loc[i, "이자보상배율"] = df2.iloc[i]['이자보상배율(이자비용)(IFRS연결)']
            #print(df.iloc[i]['이자보상배율(이자비용)(IFRS연결)'])

        # 신 회계가 결측치라면
        else:
            df2.loc[i, "이자보상배율"] = df2.iloc[i]['이자보상배율(이자비용)'] # 구 회계값 삽입
            #print(df.iloc[i]['이자보상배율(이자비용)'])

    # 구 기준 회계값에 결측치가 있다면
    else:
        # 신 회계 기준 회계값에 결측치가 없다면
        if df2.iloc[i]['이자보상배율(이자비용)(IFRS연결)'] != 77777777.77:
            # 신 회계값 삽입
            df2.loc[i, "이자보상배율"] = df2.iloc[i]['이자보상배율(이자비용)(IFRS연결)']
            #print(df.iloc[i]['이자보상배율(이자비용)(IFRS연결)'])

        # 이자보상배율에 대한 데이터가 아예 없는 경우 임의의 값 삽입
        else:
            df2.loc[i, "이자보상배율"] = 11111111.11

In [10]:
# csv 한바퀴 돌립니다
for i in range(0,len(df2)):

    #직접 계산하니까 inf, -inf는 왜 나오는 거야...

    if df2.iloc[i]['이자보상배율'] == math.inf:
        df2.loc[i, "이자보상배율"] = 11111111.11
    elif df2.iloc[i]['이자보상배율'] == -math.inf:
        df2.loc[i, "이자보상배율"] = 11111111.11

In [11]:
df2

,회사명,회계년도,영업이익(손실)(천원),이자비용(천원),영업이익(손실)(연결)(천원),이자비용(연결)(천원),[제조]영업이익증가율,[제조]재고자산증가율,[제조]자기자본증가율,[제조]매출액증가율,[제조]매출액총이익률,[제조]부가가치율,[제조]노동소득분배율,이자보상배율(이자비용),이자보상배율(이자비용)(IFRS연결),이자보상배율
0,(주)CMG제약,2000/12,1534583.0,307112.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.996819,7.777778e+07,4.996819
1,(주)CMG제약,2001/12,294030.0,284906.0,NaN,NaN,0.00,0.00,0.00,0.00,5.53,11.57,65.34,1.032025,7.777778e+07,1.032025
2,(주)CMG제약,2002/12,-3817019.0,382103.0,NaN,NaN,0.00,-39.91,-25.65,66.81,-10.12,0.46,2111.99,-9.989503,7.777778e+07,-9.989503
3,(주)CMG제약,2003/12,-914055.0,271348.0,NaN,NaN,0.00,-11.75,-1.55,-34.12,0.70,10.69,63.75,-3.368571,7.777778e+07,-3.368571
4,(주)CMG제약,2004/12,-2838322.0,123816.0,NaN,NaN,0.00,-53.72,-60.20,-24.76,-14.51,-15.02,0.00,-22.923709,7.777778e+07,-22.923709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32966,흥아해운(주),2015/12,16258441.0,20637952.0,21244278.0,20754125.0,2.53,-12.92,5.12,1.56,4.73,7.33,25.11,0.787793,1.023617e+00,1.023617
32967,흥아해운(주),2016/12,1154122.0,27547679.0,5895873.0,27640109.0,-92.90,56.39,13.60,-2.69,2.95,6.07,31.80,0.041895,2.133086e-01,0.213309
32968,흥아해운(주),2017/12,-18649057.0,30806949.0,-13098624.0,30888216.0,0.00,11.18,-41.46,0.11,0.36,0.43,444.12,-0.605352,-4.240654e-01,-0.424065
32969,흥아해운(주),2018/12,-40598236.0,32230077.0,-37595967.0,32310603.0,0.00,-5.92,-47.28,-9.71,-2.77,-5.34,0.00,-1.259638,-1.163580e+00,-1.163580


In [12]:
# 우리가 앞으로 작업을 엄청나게 복잡하게 할 걸 생각하면
# 중간중간 결과들을 데이터프레임으로 만들고 빼주는 게 좋습니다
df_2 = df2.drop(columns=['영업이익(손실)(천원)','이자비용(천원)','영업이익(손실)(연결)(천원)','이자비용(연결)(천원)'])
df_2

,회사명,회계년도,[제조]영업이익증가율,[제조]재고자산증가율,[제조]자기자본증가율,[제조]매출액증가율,[제조]매출액총이익률,[제조]부가가치율,[제조]노동소득분배율,이자보상배율(이자비용),이자보상배율(이자비용)(IFRS연결),이자보상배율
0,(주)CMG제약,2000/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.996819,7.777778e+07,4.996819
1,(주)CMG제약,2001/12,0.00,0.00,0.00,0.00,5.53,11.57,65.34,1.032025,7.777778e+07,1.032025
2,(주)CMG제약,2002/12,0.00,-39.91,-25.65,66.81,-10.12,0.46,2111.99,-9.989503,7.777778e+07,-9.989503
3,(주)CMG제약,2003/12,0.00,-11.75,-1.55,-34.12,0.70,10.69,63.75,-3.368571,7.777778e+07,-3.368571
4,(주)CMG제약,2004/12,0.00,-53.72,-60.20,-24.76,-14.51,-15.02,0.00,-22.923709,7.777778e+07,-22.923709
...,...,...,...,...,...,...,...,...,...,...,...,...
32966,흥아해운(주),2015/12,2.53,-12.92,5.12,1.56,4.73,7.33,25.11,0.787793,1.023617e+00,1.023617
32967,흥아해운(주),2016/12,-92.90,56.39,13.60,-2.69,2.95,6.07,31.80,0.041895,2.133086e-01,0.213309
32968,흥아해운(주),2017/12,0.00,11.18,-41.46,0.11,0.36,0.43,444.12,-0.605352,-4.240654e-01,-0.424065
32969,흥아해운(주),2018/12,0.00,-5.92,-47.28,-9.71,-2.77,-5.34,0.00,-1.259638,-1.163580e+00,-1.163580


In [13]:
# 일단 우리의 소중한 결과물을 빼놓도록 하죠
df_2.to_csv("../csv_data/이자보상배율2차.csv", encoding="CP949")

In [14]:
# 현재 데이터 기준으로 한계기업과 4년 연속 이.보.배 1 미만인 기업을 찾을 겁니다

f_company = [] # 한계기업을 저장하는 리스트
f_company_x = [] # 4년 연속 이자보상배율 1 미만인 기업 삽입
count = 0 # 한계기업 카운트용
company = 'x' # 한계기업 기록용

# 이거도 csv 한바퀴 돌립니다
for i in range(0, len(df_2)):

    # 회사명이 이미 기록된 상태에서 또 중복으로 적을 필요는 없습니다
    if company == df_2.iloc[i]['회사명']:
        # 다만 그 기업이 한계기업인데
        if count > 3:
            # 어머? 4년 연속이네요? 기록!!!
            if df_2.iloc[i]['이자보상배율'] < 1:
                f_company_x.append(company)
                count = 0

                # 반복문에서 볼일 다 끝났으니 통과합니다
                continue
        
        # 그게 아니라면 반복문을 통과합니다
        continue

    # 이자보상배율이 1 미만인 경우 카운트를 늘립니다
    if df_2.iloc[i]['이자보상배율'] < 1:
            count += 1
    # 도중에 이자보상배율이 1 미만이면 탈출합니다
    else:
        count = 0

    # 한계 기업 클럽에 오셨습니다. 환영합니다.
    if count == 3:
        company = df_2.iloc[i]['회사명']
        f_company.append(company)
        count += 1

In [ ]:
f_company

In [ ]:
f_company_x

In [17]:
# 분명히 이거보다 더 좋은 방법이 있을건데
# 맷돌이 굴러가질 않네요

# 이건 f_company 리스트에 들어잇는 데이터와 일치하면 True, 아니면 False를 반환합니다
df_4 = df_2.isin(f_company)
df_4

,회사명,회계년도,[제조]영업이익증가율,[제조]재고자산증가율,[제조]자기자본증가율,[제조]매출액증가율,[제조]매출액총이익률,[제조]부가가치율,[제조]노동소득분배율,이자보상배율(이자비용),이자보상배율(이자비용)(IFRS연결),이자보상배율
0,True,False,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
32966,True,False,False,False,False,False,False,False,False,False,False,False
32967,True,False,False,False,False,False,False,False,False,False,False,False
32968,True,False,False,False,False,False,False,False,False,False,False,False
32969,True,False,False,False,False,False,False,False,False,False,False,False


In [18]:
df_5 = [] # 임시 저장용 리스트

# 그렇게 만든 데이터프레임을 기존과 비교해서 True인 애들만 뽑아옵니다
for i in range(0, len(df_4)-1):
    if df_4.iloc[i]["회사명"] == True:
        df_5.append(df_2.loc[i])

In [19]:
# 그리고 저장을 합니다
df_5_frame = pd.DataFrame(df_5).reset_index(drop=True)
df_5_frame.to_csv("../csv_data/한계기업후보2.csv", encoding="CP949")

In [20]:
# 4년 연속 이자 보상 배율이 1 미만인 기업들도 똑같이 해봅니다

df_6 = df_2.isin(f_company_x)
df_7 = []

for i in range(0, len(df_6)-1):
    if df_6.iloc[i]["회사명"] == True:
        df_7.append(df_2.loc[i])

df_7_frame = pd.DataFrame(df_7).reset_index(drop=True)

In [21]:
# 중간중간 저장은 선택이 아닌 필수
df_7_frame.to_csv("../csv_data/회생불가2차.csv", encoding="CP949")

In [22]:
# 그래서 4년 연속인 애들이 얼마나 될까요? 밑에 숫자를 봅시다
df_7_frame.copy().drop_duplicates(['회사명'])

,회사명,회계년도,[제조]영업이익증가율,[제조]재고자산증가율,[제조]자기자본증가율,[제조]매출액증가율,[제조]매출액총이익률,[제조]부가가치율,[제조]노동소득분배율,이자보상배율(이자비용),이자보상배율(이자비용)(IFRS연결),이자보상배율
0,(주)CMG제약,2000/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.996819,7.777778e+07,4.996819
20,(주)DB하이텍,2000/12,13.65,-3.39,0.50,44.94,24.39,16.29,34.82,1.005895,1.019041e+00,1.019041
40,(주)ES큐브,2001/06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.188603,7.777778e+07,3.188603
59,(주)HJ중공업,2007/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.903368,8.630151e-01,0.863015
72,(주)SH에너지화학,2000/06,0.00,0.00,0.00,0.00,16.90,14.69,28.47,0.758282,7.493101e-01,0.749310
...,...,...,...,...,...,...,...,...,...,...,...,...
11830,현대바이오사이언스(주),2000/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.512496,7.777778e+07,2.512496
11850,현대에이치티(주),2000/12,0.00,0.00,0.00,0.00,33.69,23.18,54.10,9843.760684,7.777778e+07,9843.760684
11870,현대코퍼레이션(주),2000/12,24.56,686.38,-27.59,8.24,0.49,-0.07,0.00,1.563784,9.461175e-01,0.946118
11890,휴림로봇(주),2003/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.469349,7.777778e+07,4.469349


In [23]:
# 한계기업 후보 기업의 숫자는 밑에 카운트를 봅시다
df_5_frame.copy().drop_duplicates(['회사명'])

,회사명,회계년도,[제조]영업이익증가율,[제조]재고자산증가율,[제조]자기자본증가율,[제조]매출액증가율,[제조]매출액총이익률,[제조]부가가치율,[제조]노동소득분배율,이자보상배율(이자비용),이자보상배율(이자비용)(IFRS연결),이자보상배율
0,(주)CMG제약,2000/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.996819,7.777778e+07,4.996819
20,(주)DB하이텍,2000/12,13.65,-3.39,0.50,44.94,24.39,16.29,34.82,1.005895,1.019041e+00,1.019041
40,(주)ES큐브,2001/06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.188603,7.777778e+07,3.188603
59,(주)HJ중공업,2007/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.903368,8.630151e-01,0.863015
72,(주)SH에너지화학,2000/06,0.00,0.00,0.00,0.00,16.90,14.69,28.47,0.758282,7.493101e-01,0.749310
...,...,...,...,...,...,...,...,...,...,...,...,...
13646,현대코퍼레이션(주),2000/12,24.56,686.38,-27.59,8.24,0.49,-0.07,0.00,1.563784,9.461175e-01,0.946118
13666,현우산업(주),2001/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.838747,7.777778e+07,1.838747
13685,화성산업(주),2000/12,-27.81,-26.11,0.75,-9.47,18.84,13.58,37.16,1.256519,1.271813e+00,1.271813
13705,휴림로봇(주),2003/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.469349,7.777778e+07,4.469349


In [24]:
# 한계기업 후보 리스트가 어쨌든 메인이고
# 그 안에서 우리는 부실기업을 찾자는 것이고
df_5_frame['부실기업'] = 0

In [35]:
df_9 = df_5_frame.isin(df_7_frame)

In [36]:
df_9

,회사명,회계년도,[제조]영업이익증가율,[제조]재고자산증가율,[제조]자기자본증가율,[제조]매출액증가율,[제조]매출액총이익률,[제조]부가가치율,[제조]노동소득분배율,이자보상배율(이자비용),이자보상배율(이자비용)(IFRS연결),이자보상배율,한계기업
0,True,True,False,False,False,False,False,False,False,True,True,True,False
1,True,True,True,True,True,True,True,True,True,True,True,True,False
2,True,True,True,True,True,True,True,True,True,True,True,True,False
3,True,True,True,True,True,True,True,True,True,True,True,True,False
4,True,True,True,True,True,True,True,True,True,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13736,False,False,False,False,False,False,False,False,False,False,False,False,False
13737,False,False,False,False,False,False,False,False,False,False,False,False,False
13738,False,False,False,False,False,False,False,False,False,False,False,False,False
13739,False,False,False,False,False,False,False,False,False,False,False,False,False


In [39]:
# 여기서 4년 연속이면 1, 아니면 0

for i in range(0, len(df_5_frame)):
    if df_9.iloc[i]['회사명'] == True:
        df_5_frame.loc[i, '부실기업'] = 1

In [40]:
df_5_frame

,회사명,회계년도,[제조]영업이익증가율,[제조]재고자산증가율,[제조]자기자본증가율,[제조]매출액증가율,[제조]매출액총이익률,[제조]부가가치율,[제조]노동소득분배율,이자보상배율(이자비용),이자보상배율(이자비용)(IFRS연결),이자보상배율,한계기업
0,(주)CMG제약,2000/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.996819,7.777778e+07,4.996819,1
1,(주)CMG제약,2001/12,0.00,0.00,0.00,0.00,5.53,11.57,65.34,1.032025,7.777778e+07,1.032025,1
2,(주)CMG제약,2002/12,0.00,-39.91,-25.65,66.81,-10.12,0.46,2111.99,-9.989503,7.777778e+07,-9.989503,1
3,(주)CMG제약,2003/12,0.00,-11.75,-1.55,-34.12,0.70,10.69,63.75,-3.368571,7.777778e+07,-3.368571,1
4,(주)CMG제약,2004/12,0.00,-53.72,-60.20,-24.76,-14.51,-15.02,0.00,-22.923709,7.777778e+07,-22.923709,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13736,흥아해운(주),2014/12,-5.42,-8.50,9.18,6.21,4.44,6.87,23.47,0.959222,1.115251e+00,1.115251,0
13737,흥아해운(주),2015/12,2.53,-12.92,5.12,1.56,4.73,7.33,25.11,0.787793,1.023617e+00,1.023617,0
13738,흥아해운(주),2016/12,-92.90,56.39,13.60,-2.69,2.95,6.07,31.80,0.041895,2.133086e-01,0.213309,0
13739,흥아해운(주),2017/12,0.00,11.18,-41.46,0.11,0.36,0.43,444.12,-0.605352,-4.240654e-01,-0.424065,0


In [41]:
# 결측치 채우는 거 분명히 뭔가 잘못됐습니다.
# 이거도 윈저라이징을 해주거나 mean, median 처리를 해줘야함.

df_5_frame['[제조]영업이익증가율'].fillna(0, inplace=True)
df_5_frame['[제조]재고자산증가율'].fillna(0, inplace=True)
df_5_frame['[제조]자기자본증가율'].fillna(0, inplace=True)
df_5_frame['[제조]매출액증가율'].fillna(0, inplace=True)
df_5_frame['[제조]매출액총이익률'].fillna(0, inplace=True)
df_5_frame['[제조]부가가치율'].fillna(0, inplace=True)
df_5_frame['[제조]노동소득분배율'].fillna(0, inplace=True)

In [43]:
df_5_frame.corr()

,[제조]영업이익증가율,[제조]재고자산증가율,[제조]자기자본증가율,[제조]매출액증가율,[제조]매출액총이익률,[제조]부가가치율,[제조]노동소득분배율,이자보상배율(이자비용),이자보상배율(이자비용)(IFRS연결),이자보상배율,한계기업
[제조]영업이익증가율,1.000000,-0.001275,0.003680,0.001565,0.020011,0.001029,-0.005376,-0.004874,0.005785,-0.005664,-0.007823
[제조]재고자산증가율,-0.001275,1.000000,0.007299,0.012261,0.000262,0.000209,-0.001625,-0.002478,-0.009662,-0.003913,-0.006367
[제조]자기자본증가율,0.003680,0.007299,1.000000,0.008014,-0.001903,0.001988,-0.001373,-0.003834,0.024016,-0.004384,-0.004348
[제조]매출액증가율,0.001565,0.012261,0.008014,1.000000,-0.013728,0.000200,-0.001490,-0.001650,-0.000571,-0.003722,-0.004085
[제조]매출액총이익률,0.020011,0.000262,-0.001903,-0.013728,1.000000,-0.000997,0.008797,-0.044304,0.018377,0.034167,-0.002466
[제조]부가가치율,0.001029,0.000209,0.001988,0.000200,-0.000997,1.000000,0.001297,0.000549,0.011344,-0.004026,0.001526
[제조]노동소득분배율,-0.005376,-0.001625,-0.001373,-0.001490,0.008797,0.001297,1.000000,-0.007306,0.001785,0.035807,0.003392
이자보상배율(이자비용),-0.004874,-0.002478,-0.003834,-0.001650,-0.044304,0.000549,-0.007306,1.000000,0.043706,0.837470,-0.007926
이자보상배율(이자비용)(IFRS연결),0.005785,-0.009662,0.024016,-0.000571,0.018377,0.011344,0.001785,0.043706,1.000000,0.161501,-0.041103
이자보상배율,-0.005664,-0.003913,-0.004384,-0.003722,0.034167,-0.004026,0.035807,0.837470,0.161501,1.000000,-0.006335
